In [6]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}+${CUDA}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}+{CUDA}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
# Helper function for visualization.
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

1.13.0+cpu



[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import sys
import os
import collections
from collections import defaultdict
import numpy as np
import pandas as pd
from scipy import stats
import re

In [8]:
cs_file = "./data/Final.csv"
data = pd.read_csv(cs_file)
data.shape

C:\Users\let\AppData\Local\Temp\ipykernel_64116\3657760033.py:2: DtypeWarning: Columns (5,14) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(cs_file)


(485250, 20)

In [9]:
data.columns = ['unnamed','studentId','univName', 'major', 'program', 'season', 'decision', 'Method', 'decdate', 'decdate_ts', 'cgpa', 'greV', 'greQ', 
           'greA', 'is_new_gre', 'gre_subject','status', 'post_data', 'post_timestamp', 'comments']
data.drop(data.columns[data.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
data.head()

,studentId,univName,major,program,season,decision,Method,decdate,decdate_ts,cgpa,greV,greQ,greA,is_new_gre,gre_subject,status,post_data,post_timestamp,comments
0,0,University Of Waterloo,Systems Design Engineering,MS,NaN,Accepted,Website,"(1, 7, 2019)",1.561964e+09,NaN,NaN,NaN,NaN,NaN,NaN,International,"(10, 7, 2019)",1562742000,NaN
1,1,Northeastern University,Electrical Engineering,PhD,F19,Rejected,Website,"(8, 7, 2019)",1.562569e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(10, 7, 2019)",1562742000,Got email today!
2,2,The University Of Auckland,Electrical And Electronic Engineering,MS,NaN,Accepted,Website,"(19, 6, 2019)",1.560928e+09,NaN,NaN,NaN,NaN,NaN,NaN,International,"(9, 7, 2019)",1562655600,Top Student\nIELTS: 7.5\nContacted Supervisor\...
3,3,Radford University,Counseling Psychology PsyD.,Other,F19,Accepted,Phone,"(4, 3, 2019)",1.551686e+09,NaN,NaN,NaN,NaN,NaN,NaN,American,"(9, 7, 2019)",1562655600,NaN
4,4,University Of Chittagong,Computer Science,MS,NaN,NaN,Other,"(9, 7, 2019)",1.562656e+09,3.2,163.0,168.0,4.0,True,NaN,International,"(9, 7, 2019)",1562655600,NaN


In [10]:
data = data.drop('season',1)
data = data.drop('Method',1)
data = data.drop('decdate',1)
data = data.drop('decdate_ts',1)
data = data.drop('is_new_gre',1)
data = data.drop('gre_subject',1)
data = data.drop('status',1)
data = data.drop('post_data',1)
data = data.drop('post_timestamp',1)
data = data.drop('comments',1)
data = data.dropna()
data.head()


C:\Users\let\AppData\Local\Temp\ipykernel_64116\2744927780.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  data = data.drop('season',1)
C:\Users\let\AppData\Local\Temp\ipykernel_64116\2744927780.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  data = data.drop('Method',1)
C:\Users\let\AppData\Local\Temp\ipykernel_64116\2744927780.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  data = data.drop('decdate',1)
C:\Users\let\AppData\Local\Temp\ipykernel_64116\2744927780.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  data = data.drop('decdate_ts',1)
C:\Users\let\AppData\Local\Temp\ipykernel_64116\2744927780.py:5: FutureWarning: In 

,studentId,univName,major,program,decision,cgpa,greV,greQ,greA
7,7,New York University (NYU) - Steinhardt,Communication Sciences And Disorders,MS,Accepted,3.59,157.0,147.0,4.0
14,14,Ohio State University,Chemistry,PhD,Accepted,4.00,150.0,166.0,3.0
17,17,Texas A&M University,International Affairs,MS,Accepted,3.57,157.0,151.0,5.5
33,33,Iowa State University,Computer Science,PhD,Rejected,3.90,159.0,170.0,5.0
46,46,"University Of California, Irvine",Biotechnology Management,MS,Accepted,3.66,155.0,167.0,4.0


In [11]:
data = data[(data['program'] =='MS') | (data['program'] == 'MBA') | (data['program'] == 'MFA')]

In [12]:
data.head(n=3414)

,studentId,univName,major,program,decision,cgpa,greV,greQ,greA
7,7,New York University (NYU) - Steinhardt,Communication Sciences And Disorders,MS,Accepted,3.59,157.0,147.0,4.0
17,17,Texas A&M University,International Affairs,MS,Accepted,3.57,157.0,151.0,5.5
46,46,"University Of California, Irvine",Biotechnology Management,MS,Accepted,3.66,155.0,167.0,4.0
64,64,Boston University,Psychology,MS,Accepted,3.10,161.0,157.0,4.0
70,70,New York University (NYU) Steinhardt,Speech Language Pathology (Online),MS,Accepted,3.68,149.0,157.0,3.0
...,...,...,...,...,...,...,...,...,...
31286,31286,Temple University,Speech Language Pathology,MS,Accepted,3.99,158.0,160.0,4.5
31289,31289,University Of Delaware (UD),Speech Language Pathology,MS,Accepted,3.97,158.0,160.0,4.5
31292,31292,University Of Illinois,Speech Language Pathology,MBA,Accepted,3.69,151.0,154.0,4.0
31308,31308,Technical University Of Denmark (DTU),Environmental Engineering,MS,Accepted,3.20,160.0,163.0,5.0


In [13]:
processed_data = data[['studentId','greV', 'greQ', 'greA', 'cgpa', 'univName', 'major', 'program', 'decision']]
processed_data = processed_data[processed_data['decision'] =='Accepted']
processed_data.head()

processed_data.to_csv('./data/Processed_data.csv')